In [ ]:
# loading packages

import os
import pandas as pd
import numpy as np

# plotting packages
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clrs
import seaborn as sns

# Kmeans, aglomerative clustering algorithm and silhouette_score packages from scikit-learn
os.environ["OMP_NUM_THREADS"] = '1'
from sklearn.cluster import KMeans, AgglomerativeClustering 
from scipy.cluster.hierarchy import dendrogram, linkage # For Ward's linkage criterion
from sklearn.metrics import silhouette_samples,silhouette_score

## Load raw data

In [ ]:
# load raw data
DATA_FOLDER = './'
raw = pd.read_csv(os.path.join(DATA_FOLDER, 'countryriskdata.csv'))

# check the raw data
print("Size of the dataset (row, col): ", raw.shape)
print("\nFirst 5 rows\n", raw.head(n=5))

## Simple exploratory analysis
### Print summary statistics
You may want to perform more exploratory analysis to determine which features/variables to include in your analysis.

In [ ]:
# print summary statistics
print("\nSummary statistics\n", raw.describe())
print("\nCorrelation Matrix\n", raw.corr())

In [ ]:
# tests to check for skewness of Data
plt.figure(1)
raw['Corruption'].plot(kind = 'hist', title = 'Corruption', alpha = 0.5)

plt.figure(2)
raw['Peace'].plot(kind = 'hist', title = 'Peace', alpha = 0.5)

plt.figure(3)
raw['Legal'].plot(kind = 'hist', title = 'Legal', alpha = 0.5)

plt.figure(4)
raw['GDP Growth'].plot(kind = 'hist', title = 'GDP Growth', alpha = 0.5)

plt.show()

In [ ]:
df = raw[['Corruption','Peace', 'Legal', 'GDP Growth']]
df_normalized  = (df - df.mean())/ df.std()

df_normalized.head(5)

### Do some pairwise scatter plots using seaborn

In [ ]:
sns.pairplot(df_normalized,vars=df_normalized.columns[0:], diag_kind="kde")
df_normalized.columns[0:]

## K means cluster
The following example forms two clusters using all the features provided.You may consider pre-processing the data, for example, feature selection and transformation. You should also experiment with the number of clusters, and determine how many clusters to form.

Ref. [Feature normalization.](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering)

Ref. [Determining the number of clusters in a dataset.](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set)

### K means with k=1 to k=11, printing inertia for each # of clusters

In [ ]:
X = df_normalized[['Corruption', 'Peace', 'Legal', 'GDP Growth']]

inertia = []

# Loop through a range of K values and calculate inertia for each K
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k) #, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

for k in range(0, 10):
    print(k+1, inertia[k])

    
# Plot the inertia values against the number of clusters (K)
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), inertia, marker='o', linestyle='--')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.grid()
plt.show()

In [ ]:
#Optimal Number of Clusters is 3 using Elbow Method
k = 3
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(X)

# print inertia & cluster center
print("inertia for k=3 is", kmeans.inertia_)
print("cluster centers: ", kmeans.cluster_centers_)

# take a quick look at result
y = kmeans.labels_
centers = kmeans.cluster_centers_
print("cluster labels: ", y)

### Visualize the result

In [ ]:
figs = [(0,1), (0,2), (0,3), (1,2)]
labels = ['Corruption', 'Peace', 'Legal', 'GDP Growth']

norm = clrs.Normalize(vmin=0,vmax=y.max()+0.8)
cmap =cm.viridis
for i in range(4):
    fig = plt.figure(i)
    plt.scatter(X.iloc[:,figs[i][0]], X.iloc[:,figs[i][1]], c = cmap(norm(y)), s =50)
    plt.scatter(centers[:,figs[i][0]], centers[:,figs[i][1]], c = 'black', s= 200, alpha = 0.5)
    plt.xlabel(labels[figs[i][0]])
    plt.ylabel(labels[figs[i][1]])

plt.show()

In [ ]:
figs = [(0,1), (0,2), (0,3), (1,2)]
labels = ['Corruption', 'Peace', 'Legal', 'GDP Growth']
colours = ['green','red','blue']


for i in range(4):
    fig = plt.figure(i, figsize = (10,10))
    fig_1 = figs[i][0]
    fig_2 = figs[i][1]
    plt.scatter(X.iloc[:,fig_1], X.iloc[:,fig_2], c=y, s=0 , alpha = 0)
    plt.scatter(centers[:,fig_2], centers[:,fig_2], c = 'black', s= 200, alpha = 0.5)
    for j in range(X.shape[0]):
        plt.text(X.iloc[j, fig_1], X.iloc[j, fig_2], raw['Abbrev'].iloc[j], color = colours[y[j]], weight ='semibold' , horizontalalignment='center')
                        
    plt.xlabel(labels[fig_1])
    plt.ylabel(labels[fig_2])                 
plt.show()                        

### List the result

Depending on the number of features you choose, you may also consider presenting your result using graphs.

In [ ]:
result = pd.DataFrame({'Country': raw['Country'], 'Abbrev': raw['Abbrev'], 'Label':y})
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(result.sort_values('Label'))

In [ ]:
df_normalized["cluster"] = y
sns.pairplot(df_normalized,vars=df_normalized.columns[2:-1],hue="cluster")

In [ ]:
# Silhouette Analysis 
# Initialize list to store silhouette scores
silhouette_scores = []

# Loop through a range of K values and calculate silhouette score for each K
for k in range(2, 11):  # Silhouette score requires at least 2 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print("For  # of Clusters :" , k, "The Average Silhouette Score is :" , silhouette_avg)
    
# Plot silhouette scores against the number of clusters (K)
plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), silhouette_scores, marker='o', linestyle='--')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal K')
plt.grid()
plt.show()

In [ ]:
agg_clustering = AgglomerativeClustering(n_clusters=3, linkage='ward')
cluster_labels = agg_clustering.fit_predict(X)

# Add the cluster labels to the DataFrame
raw['Cluster_agg'] = cluster_labels

# Print the countries in each cluster
for cluster_id in range(3):  # Adjust the number of clusters as needed
    cluster_data = raw[raw['Cluster_agg'] == cluster_id]
    print(f"Cluster {cluster_id + 1}:")
    print(cluster_data[['Country', 'Cluster_agg']])
    print()
    
# Dendrogram for visualization
print()
linkage_matrix = linkage(X, method='ward')
plt.figure(figsize=(14, 6))
dendrogram(linkage_matrix, orientation="top",  labels = raw['Country'].to_list())
plt.title("Dendrogram for Agglomerative Hierarchical Clustering with Ward's Linkage")
plt.xlabel("Countries")
plt.ylabel("Euclidean Distance")
plt.show()